In [0]:
import os, glob
import glob
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image
from keras.datasets import mnist
from PIL import Image as pil_image

model = InceptionV3() # Load a model and its weights

def resize_mnist(x):
    x_list = []
    for i in range(x.shape[0]):
        img = image.array_to_img(x[i, :, :, :].reshape(28, 28, -1))
        #img.save("mnist-{0:03d}.png".format(i))
        img = img.resize(size=(299, 299), resample=pil_image.LANCZOS)
        x_list.append(image.img_to_array(img))
    return np.array(x_list)

def resize_do_nothing(x):
    return x

def inception_score(x, resizer, batch_size=32,splits=10):
    r = None
    n_batch = (x.shape[0]+batch_size-1) // batch_size
    for j in range(n_batch):
        x_batch = resizer(x[j*batch_size:(j+1)*batch_size, :, :, :])
        r_batch = model.predict(preprocess_input(x_batch)) # r has the probabilities for all classes
        r = r_batch if r is None else np.concatenate([r, r_batch], axis=0) # r is N*1
    scores = []
    for i in range(splits):
      print('computing IS for split ', splits)
      part_y = r[(i * r.shape[0] // splits):((i + 1) * r.shape[0] // splits), :]
      p_y = np.mean(part_y, axis=0) # p(y)
      e = part_y*np.log(part_y/p_y) # p(y|x)log(P(y|x)/P(y))
      e = np.sum(e, axis=1) # KL(x) = Σ_y p(y|x)log(P(y|x)/P(y))
      e = np.mean(e, axis=0)
      scores.append(np.exp(e))
    
    return np.mean(scores), np.std(scores) # Inception score

def mnist_inception_score(n_train):
    (x_train, y_train), (x_val, y_val) = mnist.load_data()
    x_train = np.expand_dims(x_train, axis=3) # shape=(60000, 28, 28) --> (60000, 28, 28, 1)
    x_train = np.tile(x_train, (1, 1, 1, 3)) # shape=(60000, 28, 28, 1) --> (60000, 28, 28, 3)
    return inception_score(x_train[0:n_train, :, :, :], resize_mnist)

print("Inception score (MNIST, 32):", mnist_inception_score(600))